In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

from skimage.filters import threshold_otsu
from skimage import morphology


In [2]:

def normalize(img):
    """Normalize image"""

    im = np.copy(img).astype(np.float32)
    norm_im = (img - np.min(im)) / (np.max(im) - np.min(im))
    norm_im = (norm_im * 255).astype(np.uint8)

    return norm_im

def otsu(img):
    t = threshold_otsu(img)
    retval = np.zeros_like(img).astype(np.uint8)
    retval[img > t] = 255
    return retval

def invert_uint8(img):
    retval = np.zeros_like(img)
    retval[img == 0] = 255
    return retval


def box_overlay(img, left, right, top, bottom):

    overlay = np.zeros_like(img)
    overlay[top:bottom, left] = [0, 0, 255]
    overlay[top:bottom, right] = [0, 0, 255]
    overlay[top, left:right] = [0, 0, 255]
    overlay[bottom, left:right] = [0, 0, 255]

    return overlay

In [3]:
class Object():
    
    def __init__(self, center, bbox, bbox_color):
        """
        Object class constructor

        Parameters
        ----------
        position : [tuple(int x, int y)]
        bbox : [tuple(int top, int left, int bottom, int right)]

        Returns
        -------
        None.

        """
        self._center = center
        self._bbox = bbox
        self._bbox_color = bbox_color

    def get_bbox_mask(self, in_img):

        mask = np.zeros_like(in_img, dtype=np.uint8)
        mask[self._bbox[0]:self._bbox[2], self._bbox[1]] = self._bbox_color
        mask[self._bbox[0]:self._bbox[2], self._bbox[3]] = self._bbox_color
        mask[self._bbox[0], self._bbox[1]:self._bbox[3]] = self._bbox_color
        mask[self._bbox[2], self._bbox[1]:self._bbox[3]] = self._bbox_color

        return mask

In [4]:
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.color import label2rgb

In [5]:
vid = cv2.VideoCapture(0)

while(True):

    ret, frame = vid.read()

    norm = normalize(frame)
    gray = cv2.cvtColor(norm, cv2.COLOR_BGR2GRAY)
    bin = invert_uint8(otsu(gray))
    cleared = clear_border(bin)
    label_image = label(cleared)
    image_label_overlay = label2rgb(label_image, image=frame, bg_label=0)
    
    obj_lst = []
    for region in regionprops(label_image):
        if region.area >= 100:
            obj_lst.append(
                Object(
                    center=region.centroid,
                    bbox=region.bbox,
                    bbox_color=[0,0,255]
                )
            )

    if len(obj_lst) == 0:
        overlayed = frame.copy()
    else:
        for i, obj in enumerate(obj_lst):
            if i == 0:
                overlayed = cv2.addWeighted(
                    src1=frame,
                    alpha=1,
                    src2=obj.get_bbox_mask(frame),
                    beta=0.5,
                    gamma=0
                )
            else:
                overlayed = cv2.addWeighted(
                    src1=overlayed,
                    alpha=1,
                    src2=obj.get_bbox_mask(frame),
                    beta=0.5,
                    gamma=0
                )


    cv2.imshow("frame", frame)
    cv2.imshow("processed", overlayed)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

vid.release()
cv2.destroyAllWindows()

qt.qpa.plugin: Could not find the Qt platform plugin "wayland" in "/home/maxime/mambaforge/envs/test/lib/python3.11/site-packages/cv2/qt/plugins"
